In [21]:
import os

import numpy as np
import pandas as pd

from os.path import abspath

# Task 0
Read the dataset from csv file & perform data cleaning - remove all rows, which contains `?` in some columns.
Also check for data correctness (salary & salary $K).

In [22]:

data = pd.read_csv(abspath(r"E:\git\py-adult-data-analysis\data\adult.csv"))
df = pd.DataFrame(data)
df.head()

,Unnamed: 0,age,workclass,education,marital-status,occupation,relationship,race,sex,hours-per-week,native-country,salary,salary K$
0,0,39,State-gov,Bachelors,Never-married,Adm-clerical,Not-in-family,White,Male,40,United-States,<=50K,39
1,1,50,Self-emp-not-inc,Bachelors,Married-civ-spouse,Exec-managerial,Husband,White,Male,13,United-States,<=50K,35
2,2,38,Private,HS-grad,Divorced,Handlers-cleaners,Not-in-family,White,Male,40,United-States,<=50K,27
3,3,53,Private,11th,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,40,United-States,<=50K,43
4,4,28,Private,Bachelors,Married-civ-spouse,Prof-specialty,Wife,Black,Female,40,Cuba,<=50K,25


In [23]:
df_cleaned = df.loc[(df != "?").all(axis=1)]
df_cleaned.head()

,Unnamed: 0,age,workclass,education,marital-status,occupation,relationship,race,sex,hours-per-week,native-country,salary,salary K$
0,0,39,State-gov,Bachelors,Never-married,Adm-clerical,Not-in-family,White,Male,40,United-States,<=50K,39
1,1,50,Self-emp-not-inc,Bachelors,Married-civ-spouse,Exec-managerial,Husband,White,Male,13,United-States,<=50K,35
2,2,38,Private,HS-grad,Divorced,Handlers-cleaners,Not-in-family,White,Male,40,United-States,<=50K,27
3,3,53,Private,11th,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,40,United-States,<=50K,43
4,4,28,Private,Bachelors,Married-civ-spouse,Prof-specialty,Wife,Black,Female,40,Cuba,<=50K,25


In [37]:
df = df_cleaned
condition_1 = (df["salary"] == "<=50K") & (df["salary K$"] <= 50)
condition_2 = (df["salary"] == ">50K") & (df["salary K$"] > 50)

df["salary_check"] = (condition_1 | condition_2)

# Check data correctness for salary and salary $K  
(df["salary_check"] == True).all()

df.sample(10)

,Unnamed: 0,age,workclass,education,marital-status,occupation,relationship,race,sex,hours-per-week,native-country,salary,salary K$,salary_check
26805,26805,37,2,11,Not Married,0,Other-relative,White,0,40,4,<=50K,37,True
17705,17705,31,2,9,Married,3,Wife,White,0,40,0,<=50K,46,True
577,577,36,1,6,Not Married,5,Unmarried,White,0,35,0,<=50K,17,True
32035,32035,40,2,13,Married,4,Husband,White,1,40,4,<=50K,32,True
19344,19344,25,1,1,Not Married,5,Unmarried,Asian-Pac-Islander,0,40,0,<=50K,15,True
4373,4373,35,2,12,Not Married,0,Own-child,White,1,40,0,<=50K,45,True
19678,19678,41,2,5,Not Married,0,Unmarried,White,0,38,0,<=50K,19,True
20801,20801,43,5,0,Married,3,Husband,White,1,60,0,<=50K,22,True
8776,8776,55,5,1,Married,1,Husband,White,1,60,0,<=50K,39,True
26733,26733,24,4,0,Not Married,3,Not-in-family,White,1,50,0,<=50K,44,True


# Task 1
Print the count of men and women in the dataset.

In [25]:
df.sex.value_counts()

Male      20380
Female     9782
Name: sex, dtype: int64

# Task 2
Find the average age of men in dataset

In [26]:
average_age = df[(df["sex"] == "Male")].age.mean()
average_age

39.18400392541707

# Task 3
Get the percentage of people from Poland (native-country)

In [27]:
df["native-country"].value_counts(normalize=True)["Poland"] * 100

0.18566408063125786

# Task 4
Get the mean and standard deviation of the age for people who earn > 50K per year. After this, get it for those who earn <= 50K.

In [28]:
df.groupby("salary").age.agg(["mean", "std"])

,mean,std
salary,,
<=50K,36.60806,13.464631
>50K,43.95911,10.269633


# Task 5
Check, if there are some people without higher education (education: Bachelors, Prof-school, Assoc-acdm, Assoc-voc, Masters, Doctorate), but with > 50K salary

In [29]:
education = ["Bachelors", "Prof-school", "Assoc-acdm", "Assoc-voc", "Masters", "Doctorate"]
selection = df[~df.education.isin(education) & (df.salary == ">50K")]
selection.sample(10)

,Unnamed: 0,age,workclass,education,marital-status,occupation,relationship,race,sex,hours-per-week,native-country,salary,salary K$,salary_check
6025,6025,29,Private,HS-grad,Married-civ-spouse,Prof-specialty,Husband,White,Male,40,United-States,>50K,278,True
26073,26073,51,Self-emp-not-inc,HS-grad,Married-civ-spouse,Sales,Husband,White,Male,40,United-States,>50K,212,True
2660,2660,48,Private,HS-grad,Married-civ-spouse,Craft-repair,Husband,White,Male,45,United-States,>50K,323,True
22332,22332,36,Private,Some-college,Married-civ-spouse,Craft-repair,Husband,White,Male,40,Canada,>50K,251,True
18899,18899,58,Self-emp-not-inc,HS-grad,Married-civ-spouse,Exec-managerial,Wife,White,Female,10,United-States,>50K,271,True
18050,18050,36,Private,HS-grad,Married-civ-spouse,Adm-clerical,Wife,White,Female,40,United-States,>50K,163,True
4176,4176,54,Local-gov,HS-grad,Married-civ-spouse,Craft-repair,Husband,White,Male,40,United-States,>50K,133,True
12508,12508,37,Self-emp-inc,Some-college,Married-civ-spouse,Exec-managerial,Husband,White,Male,60,United-States,>50K,72,True
28059,28059,40,Private,HS-grad,Married-civ-spouse,Sales,Wife,White,Female,40,United-States,>50K,290,True
20055,20055,50,Private,HS-grad,Married-civ-spouse,Prof-specialty,Husband,White,Male,40,United-States,>50K,97,True


# Task 6
Get the statistics of age for each type of education. Use `groupby` and `describe` for this.

In [30]:
df.groupby("education").age.describe()

,count,mean,std,min,25%,50%,75%,max
education,,,,,,,,
10th,820.0,37.897561,16.225795,17.0,23.0,36.0,52.0,90.0
11th,1048.0,32.363550,15.089307,17.0,18.0,28.5,43.0,90.0
12th,377.0,32.013263,14.373710,17.0,19.0,28.0,41.0,79.0
1st-4th,151.0,44.622517,14.929051,19.0,33.0,44.0,56.0,81.0
5th-6th,288.0,41.649306,14.754622,17.0,28.0,41.0,53.0,82.0
7th-8th,557.0,47.631957,15.737479,17.0,34.0,49.0,60.0,90.0
9th,455.0,40.303297,15.335754,17.0,28.0,38.0,53.0,90.0
Assoc-acdm,1008.0,37.286706,10.509755,19.0,29.0,36.0,44.0,90.0
Assoc-voc,1307.0,38.246366,11.181253,19.0,30.0,37.0,45.0,84.0


# Task 7
Compare the married and non-married men salaries. Who earns more? (>50K or <=50K)
Married men are those, whom `marital-status` starts with "Married". Others are not.

In [31]:
df["marital-status"] = df["marital-status"].apply(lambda x: "Married" if x.startswith("Married") else "Not Married")

df.groupby(["marital-status", "salary"]).salary.count()

marital-status  salary
Married         <=50K      8016
                >50K       6440
Not Married     <=50K     14638
                >50K       1068
Name: salary, dtype: int64

# Task 8
Get the max hours per week some person works. How many people works the same amount of hours per week?

In [32]:
max_hours = df["hours-per-week"].max()
max_hours


99

In [33]:
(df["hours-per-week"] == max_hours).sum()

78

# Task 9
Analyze the correlation between data in dataset. Understand connected fields in it and print highlight thier connection.

In [34]:
df.head()

,Unnamed: 0,age,workclass,education,marital-status,occupation,relationship,race,sex,hours-per-week,native-country,salary,salary K$,salary_check
0,0,39,State-gov,Bachelors,Not Married,Adm-clerical,Not-in-family,White,Male,40,United-States,<=50K,39,True
1,1,50,Self-emp-not-inc,Bachelors,Married,Exec-managerial,Husband,White,Male,13,United-States,<=50K,35,True
2,2,38,Private,HS-grad,Not Married,Handlers-cleaners,Not-in-family,White,Male,40,United-States,<=50K,27,True
3,3,53,Private,11th,Married,Handlers-cleaners,Husband,Black,Male,40,United-States,<=50K,43,True
4,4,28,Private,Bachelors,Married,Prof-specialty,Wife,Black,Female,40,Cuba,<=50K,25,True


In [39]:

def map_and_replace(column):
    mapper = {value: num for num, value in enumerate(column.unique())}
    return column.map(mapper)

# Apply mapping and replacement
df["workclass"] = map_and_replace(df["workclass"])
df["native-country"] = map_and_replace(df["native-country"])
df["education"] = map_and_replace(df["education"])
df["occupation"] = map_and_replace(df["occupation"])

# Map 'Male' to 1 and 'Female' to 0 in the 'sex' column
df["sex"] = df["sex"].map({"Male": 1, "Female": 0})
df


,Unnamed: 0,age,workclass,education,marital-status,occupation,relationship,race,sex,hours-per-week,native-country,salary,salary K$,salary_check
0,0,39,0,0,Not Married,0,Not-in-family,White,NaN,40,0,<=50K,39,True
1,1,50,1,0,Married,1,Husband,White,NaN,13,0,<=50K,35,True
2,2,38,2,1,Not Married,2,Not-in-family,White,NaN,40,0,<=50K,27,True
3,3,53,2,2,Married,2,Husband,Black,NaN,40,0,<=50K,43,True
4,4,28,2,0,Married,3,Wife,Black,NaN,40,1,<=50K,25,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,32556,27,2,6,Married,9,Wife,White,NaN,38,0,<=50K,36,True
32557,32557,40,2,1,Married,8,Husband,White,NaN,40,0,>50K,173,True
32558,32558,58,2,1,Not Married,0,Unmarried,White,NaN,40,0,<=50K,40,True
32559,32559,22,2,1,Not Married,0,Own-child,White,NaN,20,0,<=50K,38,True


In [40]:
df_corr = df[["age", "workclass", "education", "occupation", "sex", "native-country", "salary K$"]].corr()
df_corr

,age,workclass,education,occupation,sex,native-country,salary K$
age,1.000000,0.064147,0.009060,0.002072,NaN,-0.005066,0.208203
workclass,0.064147,1.000000,-0.022917,-0.046620,NaN,-0.012075,0.086385
education,0.009060,-0.022917,1.000000,0.056654,NaN,0.060493,-0.039488
occupation,0.002072,-0.046620,0.056654,1.000000,NaN,0.018083,-0.064373
sex,NaN,NaN,NaN,NaN,NaN,NaN,NaN
native-country,-0.005066,-0.012075,0.060493,0.018083,NaN,1.000000,-0.020267
salary K$,0.208203,0.086385,-0.039488,-0.064373,NaN,-0.020267,1.000000
